TODO
<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/main/scripts/100_note_generation/100_GenerateAnonymousCareNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenCare AI: Synthetic Healthcare Data Generation

**Author:** Eva Rombouts  
**Date:** 2024-05-12  
**Updated:** 2024-09-01  
**Version:** 1.4

### Description
This notebook generates synthetic healthcare data for NLP experiments.  
It utilizes OpenAI's models to create a synthetic dataset with progress notes of nursing home clients for use in machine learning and data analysis.  

These notes are “anonymous,” meaning they are not based on specific client profiles or scenarios, unlike the notes generated in later steps. The notes created in this notebook

An example of the resulting [dataset](https://huggingface.co/datasets/ekrombouts/dutch_nursing_home_notes) can be found on HuggingFace.   

Version 1.1: Updated terminology - consistent use of notes in stead of reports.  
Version 1.2: Minor changes in pip installs + removed all traces of the term 'reports.  
Version 1.3: Minor changes in pip installs.  
Version 1.4: Refinements made following testing. 

I considered using LangChain, but generating multiple completions per request is not straightforward, so I stuck to the OpenAI API. 

In [2]:
# !pip install GenCareAI
from GenCareAI.GenCareAIUtils import GenCareAISetup

In [3]:
setup = GenCareAISetup()
if setup.environment == 'Colab':
        !pip install -q openai

In [4]:
import pandas as pd
from openai import OpenAI

In [4]:
# Constants / variables
SEED = 6
MODEL = 'gpt-3.5-turbo-0125'
TEMPERATURE = 1.1

NUM_COMPLETIONS = 10 # 100 Number of completions per query

FN_NOTES = setup.get_file_path('data/gcai_notes.csv')

In [5]:
# System role, describes the assistant's behavior in a specific context
SYSTEM_ROLE = '''
Je bent een specialist in het genereren van fictieve data voor natural language processing projecten in de zorg.

Je spreekt de taal van een niveau 3 verzorgende in het verpleeghuis.
'''

In [6]:
# Dictionary of user roles for different topics of the notes
USER_ROLES = {
        'ADL':
        '''
Dit zijn voorbeelden van rapportages over ADL:
- Dhr. zijn haar gewassen en zijn baard geschoren.
- Inco van mw, was verzadigd vanmorgen en bed was nat.
- Het is niet goed gegaan Mw had een ongelukje met haar kleding en defeaceren Mw was incontinent Mw geholpen met opfrissen en de kleding in de was gedaan
- U bent vanmorgen gedoucht, uw haren zijn gewassen.

Andere rapportages kunnen bijvoorbeeld gaan over: wassen, aankleden, tanden poetsen, klaarmaken voor de dag, klaarmaken voor de nacht, douchen, gebitsprothese schoonmaken of hulp na incontinentie.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        ''',
        'eten_drinken':
        '''
Dit zijn voorbeelden van rapportages over eten en drinken:
- Ik kreeg van de dagdienst door dat dhr. zich verslikt in haar drinken. Drinken verdikt aangeboden. Dit ging goed.
- Ochtendzorg verliep goed, dhr was wel zeer vermoeid. Dhr heeft goed gegeten en gedronken. Dhr is na de lunch op bed geholpen om te rusten.
- Nee ik wil niet meer. ik vond niet lekker. Mw heeft ochtend goed gegeten en gedronken. tussen de middageten mw wilde niet. zij heeft paar hapjes vla gegeten en een glas limonade gedronken.
- Mw heeft op bed een paar hapjes pap gegeten.
- De fresubin creme is niet op voorraad. mw ipv de creme fresubin drink aanbieden Fresubin komt vogende week weer binnen.

Andere rapportages kunnen bijvoorbeeld gaan over: wat de client wel of niet heeft gegeten, welke hulp nodig is bij eten (volledige hulp, aansporing, aangepast bestek of beker), verslikken, bijhouden vocht- en voedingslijst.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        ''',
        'sociaal':
        '''
Dit zijn voorbeelden van rapportages over sociale interactie en activiteiten:
- Mw. was goed gestemd vanavond en was heel gezellig aanwezig.
- U keek naar de kerkdienst op buurt 4.
- Dhr zit met verschillende medebewoners in de binnentuin.
- Ik eet samen met mijn dochter. We gaan asperges eten.
- Mw. ging haar gangetje. Ging vanmiddag naar een muziek activiteit.

Andere rapportages kunnen bijvoorbeeld gaan over: Georganiseerde activiteiten, het krijgen van bezoek, bladeren door een tijdschriftje, interactie met medebewoners.

Hou er rekening mee dat het gaat over rapportages van mensen in een verpleeghuis, met forse beperkingen, dus de sociale interactie en activiteiten zijn beperkt. Meestal betreft het gezelligheid, maar niet altijd.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl
        ''',
        'huid':
        '''
Dit zijn voorbeelden van rapportages over huid en wonden:
- ik heb jeuk op mijn rug dhr behandeld met de cetomacrogol creme
- Wat is dat allemaal? Dhr zat aan het verband om zijn arm te plukken. Wondje op arm is klein. Dhr ervaart het verband onprettig. Pleister op het wondje gedaan.
- Dhr zijn liezen zagen er rustig uit. Dhr zijn scrotum ingesmeerd met licht zinkzalf, deze was wel rood. De liezen met beschermende zalf ingesmeerd.
- Mevr. lijkt nu decubitus te ontwikkelen op haar stuit. Mevr. haar hiel verzorgd, dit zag er oke uit, klein beetje geel beslag. Dit schoongemaakt, daarna verbonden volgens plan Dit in de gaten houden.

Andere rapportages kunnen bijvoorbeeld gaan over: oedeem, decubituswonden, ontvellingen, roodheid en jeuk van de huid. Te lange nagels, smetplekken.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        ''',
        'medisch_logistiek':
        '''
Dit zijn voorbeelden van rapportages over medische zorg en familie communicatie:

- Oren van mevr zijn uitgespoten er kwam uit beide oren veel viezigheid.
- Graag Dhr morgen wegen
- Arts vragen voor brutans 5 mg besteld
- Dochter van dhr. belde. Ze gaf aan dat ze een aanbod hebben gekregen voor verblijf in een ander verpleeghuis.
- Fam wil graag een gesprek over bezoek cardioloog in het verleden. Er is iets voorgeschreven, ws doorgegeven aan vorige arts. graag contact met familie opnemen voor gesprek of telefonisch gesprek In artsenvisite bespreken

Andere rapportages kunnen bijvoorbeeld gaan over: zorgplan besprekingen, kleine medische klachten, verzoeken van familie, bestellen van medicijnen.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        ''',
        'nachten':
        '''
Dit zijn voorbeelden van rapportages over nachten en slapen:

- Mw. heeft de gehele nacht geslapen
- Mw heeft vannacht niet zo goed geslapen. Mw was veel wakker en wat onrustig. Lastig om mw af te leiden en te zorgen dat mw weer wilde slapen. Mw heeft een slechte nachtrust gehad.
- De sensor is de gehele nacht niet afgegaan bij mw
- Dhr. ging rond 23:30 uur naar bed. Heeft de hele nacht geslapen.
- Dhr. was klaarwakker en wilde uit bed en rammelde aan het bedhek. Dhr. vertelde dat hij opgehaald zou worden. Mw. heeft hem overtuigt om toch te gaan slapen en dhr. luisterde naar mw.

Rapportages kunnen bijvoorbeeld gaan over: onrust en dwalen in de nacht, lekker slapen, toiletgang in de nacht, bellen, scheef in bed liggen.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        ''',
        'onrust':
        '''
Dit zijn voorbeelden van rapportages over onrust, probleemgedrag, stemming:

- Ga opzij. Wat ben jij lelijk Mw schopte naar een andere bewoner en wilde een ander bewoner slaan. Mw een prikkelarme omgeving aangeboden.
- dhr eet de planten van tafel dhr werd begeleid door collega om het uit te spugen werd hier geagiteerd door.
- Waar is het toilet Mag ik al eten Naar zorg toe lopen, zwaaien naar de zorg om hulp.  Mw vraagt veel bevestiging van de zorg,
- Meneer is wat onrustig loopt jammerend heen en weer en zegt steeds erg moe te zijn. Heeft een trieste blijk in zijn ogen. Meneer aangeboden om naar bed te gaan, heeft hier geen rust voor.

Rapportages kunnen bijvoorbeeld gaan over: agitatie, onrust, apathie, verwardheid.
Meestal is de verwardheid subtiel, maar soms wat heftiger.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        ''',
        'symptomen':
        '''
Dit zijn voorbeelden van rapportages over ziekte en symptomen:

- Er zat iets vocht in beide voeten. Dhr had vandaag geen steunkousen aan Blijven observeren
- Urine opvangen is tot nu toe nog niet gelukt(mw heeft er steeds def bij) Vanmiddag ook geen pijn gezien alleen evt wat frustratie als iets niet soepel loopt.
- Ik heb pijn dhr gaf pijn aan aan zijn linker pink en ringvinger. Er zitten daar een soort bloedblaren al wel langer. Graag even in de gaten houden en rapporteren of dhr meer pijn krijgt.
- Dhr. had om 6u zeer veel last van slijm en een vieze smaak in zijn mond. Dhr geassisteerd met het spoelen van zijn mond.
- Erg pijnlijk bij de ADL. Morgen graag overleg met de arts over de pijnmedicatie
- Dhr is erg benauwd, klinkt vol, heeft een reutelende ademhaling.

Rapportages kunnen bijvoorbeeld gaan over: pijn, benauwdheid, misselijkheid, diarree, rugklachten, palliatieve zorg.
Meestal zijn de klachtensubtiel, maar soms heftiger.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        ''',
        'mobiliteit':
        '''
Dit zijn voorbeelden van rapportages over mobiliteit en transfers:

- Vandaag geholpen met de passieve lift. Dit ging goed.
- Veel rondgelopen vandaag. Mw vergeet steeds haar rollator.
- De banden van de rolstoel zijn zacht. Kan de fysio hier naar kijken?
- De transfers gaan steeds moeilijker. Mw hangt erg in de actieve lift. Glijdt weg. Wil graag nog met de actieve lift geholpen worden, maar dit gaat eigenlijk niet meer. @ Ergo, graag je advies

Andere rapportages kunnen bijvoorbeeld gaan over: loophulpmiddelen, de rolstoel, valgevaar, valincidenten, transfers, tilliften.
De meeste rapportages gaan over dagelijkse dingetjes, dus niet alles is een ernstig incident.

Verzin 10 van zulke rapportages. Geef alleen de rapportages terug, gescheiden door "\n--- ". Varieer met de zinsopbouw en stijl.
        '''
}


In [8]:
def generate_and_collect_notes():
    """Main function to orchestrate data generation."""

    def setup_openai_client():
        """Create an OpenAI client instance."""
        try:
            client = OpenAI(api_key=setup.get_openai_key())
            return client
        except Exception as e:
            print(f"Failed to set up OpenAI client: {str(e)}")
            return None

    def generate_healthcare_data(client, system_role, user_role, model, seed=None, temperature=1, n=1):
        """Generate synthetic healthcare data using the OpenAI API."""
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_role},
                    {"role": "user", "content": user_role}
                ],
                seed=seed,
                temperature=temperature,
                n=n
            )
            return completion
        except Exception as e:
            print(f"An error occurred during data generation: {str(e)}")
            return None

    def parse_and_clean_notes(completions):
        """Parse and clean notes from OpenAI completions."""
        notes = []
        for choice in completions.choices:
            splits = choice.message.content.split('---')
            for split in splits:
                clean_note = split.strip().lstrip('-').strip()
                if clean_note:
                    notes.append(clean_note)
        return notes

    # Begin main logic
    client = setup_openai_client()
    if not client:
        print("OpenAI client setup failed. Exiting...")
        return

    all_notes = []
    for topic, user_role in USER_ROLES.items():
        print(f"Generating notes for topic: {topic}")
        completion = generate_healthcare_data(client, SYSTEM_ROLE, user_role, MODEL, SEED, TEMPERATURE, NUM_COMPLETIONS)
        if completion:
            notes = parse_and_clean_notes(completion)
            df = pd.DataFrame(notes, columns=['note'])
            df['topic'] = topic
            all_notes.append(df)
        else:
            print(f"Failed to generate notes for topic: {topic}")

    return all_notes

all_notes = generate_and_collect_notes()
df_all_notes = pd.concat(all_notes, ignore_index=True)

Generating notes for topic: ADL
Generating notes for topic: eten_drinken
Generating notes for topic: sociaal
Generating notes for topic: huid
Generating notes for topic: medisch_logistiek
Generating notes for topic: nachten
Generating notes for topic: onrust
Generating notes for topic: symptomen
Generating notes for topic: mobiliteit


In [13]:
def save_notes(df, filename):
    """Save the DataFrame to a CSV file."""
    try:
        df.to_csv(filename, index=False)
        print(f"Notes saved successfully to {filename}")
    except Exception as e:
        print(f"Failed to save notes: {str(e)}")
    return df 
    
saved_df = save_notes(df_all_notes, FN_NOTES)

Notes saved successfully to /Users/eva/Library/CloudStorage/GoogleDrive-e.k.rombouts@gmail.com/My Drive/Colab Notebooks/GenCareAI/data/gcai_notes.csv


In [15]:
saved_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838 entries, 0 to 837
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   note    838 non-null    object
 1   topic   838 non-null    object
dtypes: object(2)
memory usage: 13.2+ KB
